bigClassAnswerRate.ipynb 를 일부 수정하여 만들었습니다.

In [1]:
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import tqdm

<IPython.core.display.Javascript object>

In [3]:
"""
train data 또는 test data 중 feature를 추가하고 싶은 데이터를 불러오면 됩니다.!
(train과 test data를 합친 data를 불러와도 됩니다!)
"""
# TODO : 원하는 데이터의 경로를 설정해주세요
raw_data = pd.read_csv("/opt/ml/input/data/FE/elapsedTime.csv")

user_id = raw_data["userID"].unique()  # 유저 아이디 저장

<IPython.core.display.Javascript object>

In [4]:
raw_data  # 원본 확인

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elapsedTime
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,3
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,8
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,7
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,11
...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,55
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,11
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,46
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,73


<IPython.core.display.Javascript object>

In [5]:
##-------------------대분류 Feature 추가--------------------------
raw_data["bigClass"] = raw_data["testId"].str[2]

<IPython.core.display.Javascript object>

In [6]:
raw_data.head()  # 대분류 추가 이후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elapsedTime,bigClass
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,3,6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,8,6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,7,6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7,6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,11,6


<IPython.core.display.Javascript object>

In [7]:
"""
acc_by_big_category
    - key : 유저 아이디
    - value : 해당 유저의 대분류별 정답률 (인덱스가 대분류를 나타낸다. ex. index=1 -> 대분류 1)
    - default value : 기본적으로 0.5라는 값이 들어간다. 
"""
acc_by_big_category = dict()
for user in user_id:
    acc_by_big_category[user] = [0.5] * 10

<IPython.core.display.Javascript object>

In [11]:
data = raw_data[raw_data["userID"] == 0]
data.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elapsedTime,bigClass
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,3,6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,8,6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,7,6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7,6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,11,6


<IPython.core.display.Javascript object>

In [26]:
for user in tqdm.tqdm(user_id):  # 모든 유저에 대해
    data = raw_data[raw_data["userID"] == user]  # 각 유저에 대한 행만 추출하여 data에 저장
    bc_l = data["bigClass"].unique()  # 행들에 있는 고유한 대분류 추출 ex) [6, 2, 8]

    for big_class in bc_l:  # 유저가 풀어본 각 대분류에 대해
        big_class_int = int(big_class)
        user_class_interaction = data[data["bigClass"] == big_class]

        big_correct = 0
        big_total = len(user_class_interaction)

        acc_by_big_category[user][big_class_int] = int(
            sum(user_class_interaction["elapsedTime"]) / big_total
        )  # 전체 데이터를 순회하면서 알게된 대분류 정답률을 저장

100%|██████████| 7442/7442 [00:22<00:00, 335.14it/s]


<IPython.core.display.Javascript object>

In [27]:
"""
acc_by_big_category_list != acc_by_big_category
: 해당 리스트는 feature를 추가하기 전에 feature 값들을 일렬로 저장하기 위해 만든 리스트입니다.
"""
acc_by_big_category_list = list()

for u, bc in tqdm.tqdm(
    zip(raw_data["userID"], raw_data["bigClass"]), total=len(raw_data["userID"])
):  # 모든 행(instance)에 대해
    acc = acc_by_big_category[u][
        int(bc)
    ]  # 해당 유저의 해당 대분류에 대한 정답률을 위에서 정의한 dictionary에서 찾아서
    acc_by_big_category_list.append(acc)  # 리스트에 저장한다.

raw_data[
    "bigClassElaspedTimeAvg"
] = acc_by_big_category_list  # 리스트에 bigClassAcc feature 추가

100%|██████████| 2526700/2526700 [00:02<00:00, 893386.40it/s]


<IPython.core.display.Javascript object>

In [28]:
raw_data  # 대분류, 대분류별 정답률 추가 이후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elapsedTime,bigClass,bigClassElaspedTimeAvg
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,3,6,36
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,8,6,36
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,7,6,36
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7,6,36
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,11,6,36
...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,55,3,55
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,11,4,43
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,46,4,43
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,73,4,43


<IPython.core.display.Javascript object>

In [29]:
raw_data = raw_data.sort_values(by=["userID", "Timestamp"])

<IPython.core.display.Javascript object>

In [30]:
# TODO : 이름과 경로 원하는 데로 변경하기
import os

os.makedirs("/opt/ml/input/data/FE", exist_ok=True)
raw_data.to_csv(
    "/opt/ml/input/data/FE/bigClassElapsedTimeAvg.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "bigClassElaspedTimeAvg",
    ],
    index=False,
)  # 최종 결과 csv로 반환

<IPython.core.display.Javascript object>